In [ ]:
from nbdev import *
%nbdev_default_export environments

Cells will be exported to pct.environments,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Environments

> Classes to create simulated environment functions.

In [ ]:
%nbdev_hide
import sys
sys.path.append("..")

In [ ]:
%nbdev_export
import gym
import math
import numpy as np
from pct.functions import BaseFunction

In [ ]:
%nbdev_export
class OpenAIGym(BaseFunction):
    "A function that creates an runs an environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    def __init__(self, env_name=None, render=False, video_wrap=False, value=0, name="gym", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        
        self.video_wrap = video_wrap
        self.env_name=env_name
        self.create_env(env_name, 4000)
        self.render = render
        self.reward = 0
        self.done = False
        self.info = {}
        
    def __call__(self, verbose=False):
        super().check_links(1)
        self.input = self.links[0].get_value()
        self.obs = self.env.step(self.input)
            
        self.value = self.obs[0]
        self.reward = self.obs[1]
        self.done = self.obs[2]
        self.info = self.obs[3]
        
        if self.render:
            self.env.render()
            
        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        config = super().get_config()
        config["env_name"] = self.env_name
        #config["values"] = self.value
        config["reward"] = self.reward
        config["done"] = self.done
        config["info"] = self.info
        
        return config
    
    def output_string(self):
        
        if isinstance(self.value, int):
            rtn = f'{round(self.value, self.decimal_places):.{self.decimal_places}f}'
        else:
            list = [f'{round(val, self.decimal_places):.{self.decimal_places}f} ' for val in self.value]
            list.append(str(self.reward))
            list.append(" ")
            list.append(str(self.done))
            list.append(" ")
            list.append(str(self.info))
            
            rtn = ''.join(list)

        return rtn

    
    def create_env(self, env_name, max_episode_steps):
        genv = gym.make(env_name)
        genv._max_episode_steps = max_episode_steps
        if self.video_wrap:
            self.env =  vid.wrap_env(genv)
        else:
            self.env = genv
            self.env.reset()
            
    def close(self):
        self.env.close()

In [ ]:
%nbdev_export
class CartPoleV1(OpenAIGym):
    "A function that creates an runs the CartPole-v1 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 1 cart_velocity
    # 0 cart_position
    # 3 pole_velocity
    # 2 pole_angle
    def __init__(self, env_name='CartPole-v1', render=False, video_wrap=False, value=0, name="gym", links=None, new_name=True, **cargs):
        super().__init__(env_name, render, video_wrap, value, name, links, new_name, **cargs)
 
    def __call__(self, verbose=False):
        super().__call__(verbose)
        
        if self.input == 1 or self.input == -1 or self.input == 0:
            pass
        else:
            raise Exception(f'OpenAIGym: Input value of {self.input} is not valid, must be 1,0 or -1.')

        self.value = np.append(self.value, self.obs[0][0]+math.sin(self.obs[0][2]))

        




In [ ]:
%nbdev_export
class PendulumV0(OpenAIGym):
    "A function that creates an runs the Pendulum-v0 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 0 cos(theta)
    # 1 sin(theta)
    # 2 theta dot
    # 3 theta +pi/-pi (added here)
    # reward - -(theta^2 + 0.1*theta_dt^2 + 0.001*action^2)

    def __init__(self, env_name='Pendulum-v0', render=False, video_wrap=False, value=0, name="gym", links=None, new_name=True, **cargs):
        
        super().__init__(env_name, render, video_wrap, value, name, links, new_name, **cargs)
        
    
        
    def __call__(self, verbose=False):
        super().__call__(verbose)
                
        pi = math.copysign(math.acos(self.obs[0][0]), self.obs[0][1])
        self.value = np.append(self.value, pi)
        
        

### OpenAI Gym
An example showing how to use an OpenAI Gym function. And how to have another function which accesses one of the values of the gym environment. 

In [ ]:
from pct.functions import Constant
from pct.functions import IndexedParameter
from pct.putils import FunctionsList
from pct.functions import Proportional

render=False 
print(render)
acrobot = OpenAIGym("Acrobot-v1", render=render)
acrobot.add_link(Constant(1))
acrobot.get_config()

False


{'type': 'OpenAIGym',
 'name': 'gym',
 'value': 0,
 'links': {0: 'constant'},
 'env_name': 'Acrobot-v1',
 'reward': 0,
 'done': False,
 'info': {}}

In [ ]:
acrobot()
print(acrobot.reward)
print(getattr(acrobot, "reward"))

-1.0
-1.0


In [ ]:
acrobot.output_string()

'0.997 -0.083 1.000 -0.004 0.072 -0.041 -1.0 False {}'

In [ ]:
acrobot.value[0]

0.9965881135941491

The IndexedParameter type retrieves a value from a linked function based upon an index.

In [ ]:
cos_angle1 = IndexedParameter(0, name="cos_angle1")
cos_angle1.add_link(acrobot)
print(cos_angle1.get_config())
cos_angle1()

{'type': 'IndexedParameter', 'name': 'cos_angle1', 'value': 0, 'links': {0: 'gym'}, 'index': 0}


0.9965881135941491

In [ ]:
acrobot.close()

In [ ]:
%nbdev_hide
FunctionsList.getInstance().get_function("cos_angle1").set_name("cos_angle2")
assert cos_angle1.get_name() == "cos_angle2"

In [ ]:
%nbdev_hide
pp = Proportional(10)
print(pp.get_config())
pp.set_property("gain", 3)
print(pp.get_config())

{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 10}
{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 3}


In [ ]:
#gui
pen = PendulumV0(render=True)
pen.add_link(Constant([1]))
print(pen.get_config())
pen.run(steps=10, verbose=True)

{'type': 'PendulumV0', 'name': 'gym1', 'value': 0, 'links': {0: 'constant1'}, 'env_name': 'Pendulum-v0', 'reward': 0, 'done': False, 'info': {}}
-0.865 -0.502 0.739 -7.123358052111961 False {} -0.851 -0.524 0.513 -6.894543250472319 False {} -0.844 -0.536 0.269 -6.732749905239168 False {} -0.844 -0.537 0.017 -6.64418982715233 False {} -0.850 -0.527 -0.235 -6.63251384186536 False {} -0.862 -0.506 -0.480 -6.698708869079416 False {} -0.880 -0.475 -0.710 -6.841021958380259 False {} -0.901 -0.434 -0.916 -7.054903988963358 False {} -0.923 -0.385 -1.092 -7.333001171968104 False {} -0.945 -0.327 -1.230 -7.665250851704046 False {} 

In [ ]:
#gui
pen.close()

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_examples.ipynb.
Converted 01_putils.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted 05_environments.ipynb.
Converted 06_architectures.ipynb.
Converted index.ipynb.
